# Prepare Experiment & Deepspeed config (**MANDATORY**)
***

In [ ]:
ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
        "offload_optimizer": {
            "device": "none",
        },
        "offload_params": {
            "device": "none"
        },
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 200,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

training_args = {
    "num_train_epochs": 4,
    "gradient_accumulation_steps": 1,
    "per_device_train_batch_size": 128,
    "per_device_eval_batch_size": 256,
    "fp16": True,
    "weight_decay": 0.0,
    "warmup_steps": 0,
    "learning_rate": 1e-5,
    "logging_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 1,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_accuracy",
    "greater_is_better": True,
}

# usually overriden by external config:
num_gpus = 1
model_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"

# ynie config: entailment is on 0, neutral on 1 and contrad. on 2
# ours is 0: contrad., 1 entail, so we need to map
class_map = {0:2, 1:0, 2:1}
block_size = 128
logdir = "data/models/textual_entailment/ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli/"
polarity_model_dir = "data/models/polarity/bert-base-uncased/bs128_lr_1e-05/"

override_logdir = True

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
from datasets import load_dataset
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import Trainer, TrainingArguments

# Tokenize the dataset
***

In [ ]:
if "Eleuther" in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='<|startoftext|>', 
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def load_social_chem101():
    a = pd.read_csv("data/social-chem-101/social-chem-101.v1.0.tsv", sep="\t")
    return a

social_chem = load_social_chem101()
social_chem = social_chem[social_chem["split"] == "train"]
social_chem = social_chem.dropna(subset=["rot-categorization", "rot-judgment", "action", "rot-agree", "action-moral-judgment"])
social_chem = social_chem[social_chem["rot-agree"] >= 3.0]
social_chem = social_chem[social_chem["rot-bad"] == 0]
social_chem = social_chem[social_chem["rot-categorization"].apply(lambda x: "morality-ethics" in x or "social-norms" in x)]
social_chem = social_chem[social_chem["rot-judgment"].apply(lambda x: "{" not in x)]

In [ ]:
# we need the rot-action from social chemistry
def to_entailment_task(t):
    t = t.merge(social_chem[["rot","action","action-moral-judgment"]], suffixes=("","_norm"), left_on="norm", right_on="rot")
    t = t.groupby(["ID"]).nth(0)
    t["polarity"] = (t["action-moral-judgment"] > 0).astype("int")
    t["action_polarity"] = t["label"]
    # 1: entailing, 0: contradicting
    t["label"] = (t["polarity"] == t["action_polarity"]).astype("int").apply(class_map.get)
    return t


def load_action_norm_split(path):
    train, dev, test = [pd.read_json(f"{path}{x}.jsonl", lines=True) for x in ["train", "dev", "test"]]

    # construct dataframes that can actually be used
    assign_action = lambda x: x["moral_action"] if x["label"] == 1 else x["immoral_action"]
    train["action"] = train.apply(assign_action, axis=1)
    dev["action"] = dev.apply(assign_action, axis=1)
    test["action"] = test.apply(assign_action, axis=1)
    return train, dev, test

# used for testing
train, dev, test = load_action_norm_split("data/contrastive_moral_stories/original_ms/action+norm/norm_distance/")
opt_train, opt_dev, opt_test = load_action_norm_split("data/contrastive_moral_stories/optional_ms/action+norm/norm_distance/")
anti_train, anti_dev, anti_test = load_action_norm_split("data/contrastive_moral_stories/anti_ms/action+norm/norm_distance/")
#contra_train, contra_dev, contra_test = load_action_norm_split("data/contrastive_moral_stories/contra_ms/action+norm/norm_distance/")

In [ ]:
train_te, dev_te, test_te = [to_entailment_task(x) for x in [train, dev, test]]

# Train model
***

In [ ]:
dataset = datasets.DatasetDict()
dataset["train"] = datasets.Dataset.from_pandas(train_te).shuffle()
dataset["dev"] = datasets.Dataset.from_pandas(dev_te)
dataset["test"] = datasets.Dataset.from_pandas(test_te)

In [ ]:
def tokenize(samples):
    return tokenizer(samples["action"],samples["action_norm"], truncation=True, padding="max_length", max_length=block_size)

tokenized_data = dataset.map(tokenize, batched=True, batch_size=10000)

# Load the model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.resize_token_embeddings(len(tokenizer))

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

# Prepare Trainer
***

In [ ]:
# TODO: move down to training section
training_args = TrainingArguments(
    output_dir=logdir,
    overwrite_output_dir=override_logdir,
    logging_dir=logdir,
    #deepspeed= logdir + "/ds_config.json",
    report_to="tensorboard",
    **training_args
)

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print(predictions)
    y = [class_map[x] for x in predictions[:,0] > predictions[:,2]]
    return metric.compute(predictions=y, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["dev"],
    compute_metrics=compute_metrics,
)

In [ ]:
r = trainer.predict(tokenized_data["test"])
y_pred_no_ft = np.array([x for x in r.predictions[:,0] > r.predictions[:,2]], "int")
test_te["te_pred_no_ft"] = y_pred_no_ft

In [ ]:
trainer.train()

In [ ]:
r = trainer.predict(tokenized_data["test"])
y_pred_ft = np.array([x for x in r.predictions[:,0] > r.predictions[:,2]], "int")
test_te["te_pred_ft"] = y_pred_ft

In [ ]:
# save the predictions in model dir
test_te.to_csv(logdir + "original_ms_te_applied.csv")